In [ ]:
!pip install shap

In [ ]:
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd


In [ ]:
import joblib

# Load the trained model
fraud_rf = joblib.load("/content/fraud_rf.pkl")

print("Model loaded successfully!")


### Feature Importance Baseline

In [ ]:
# Get feature names from your pipeline
feature_names = fraud_rf.named_steps["pre"].get_feature_names_out()

# Extract feature importances
importances = fraud_rf.named_steps["clf"].feature_importances_
indices = np.argsort(importances)[::-1][:10]

# Plot top 10 features
plt.figure(figsize=(8,6))
sns.barplot(x=importances[indices], y=feature_names[indices])
plt.title("Top 10 Feature Importances (Random Forest)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

fraud_df = pd.read_csv("/content/fraud_engineered.csv")
fraud_feat_cols_num = [
    "purchase_value", "age", "hour_of_day", "day_of_week",
    "time_since_signup_hours", "user_txn_count_24h",
    "user_txn_sum_24h", "user_txn_count_6h"
]
fraud_feat_cols_cat = ["source", "browser", "sex"]
if "ip_country" in fraud_df.columns:
    fraud_feat_cols_cat.append("ip_country")

X_fraud = fraud_df[fraud_feat_cols_num + fraud_feat_cols_cat]
y_fraud = fraud_df["class"].astype(int)

# Stratified split
Xf_train, Xf_test, yf_train, yf_test = train_test_split(
    X_fraud, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud
)


### SHAP Force Plots (Local Explanations)

In [ ]:
rf_clf = fraud_rf.named_steps["clf"]
X_transformed = fraud_rf.named_steps["pre"].transform(Xf_test)
feature_names = fraud_rf.named_steps["pre"].get_feature_names_out()


In [ ]:
import shap

rf_clf = fraud_rf.named_steps["clf"]
X_transformed = fraud_rf.named_steps["pre"].transform(Xf_test)
feature_names = fraud_rf.named_steps["pre"].get_feature_names_out()

# Build explainer
explainer = shap.TreeExplainer(rf_clf)

# Compute SHAP values for the full test set
shap_values = explainer.shap_values(X_transformed)

print("SHAP shape:", shap_values[1].shape)   # should now be (30223, 18)
print("Data shape:", X_transformed.shape)    # (30223, 18)

# Plot summary
shap.summary_plot(shap_values[1], X_transformed, feature_names=feature_names)


In [ ]:
# Identify indices for TP, FP, FN
tp_index = np.where((yf_test==1) & (fraud_rf.predict(Xf_test)==1))[0][0]
fp_index = np.where((yf_test==0) & (fraud_rf.predict(Xf_test)==1))[0][0]
fn_index = np.where((yf_test==1) & (fraud_rf.predict(Xf_test)==0))[0][0]
shap.initjs()

# Force plots
shap.force_plot(explainer.expected_value[1], shap_values[1][tp_index],
                X_transformed[tp_index], feature_names=feature_names)

shap.force_plot(explainer.expected_value[1], shap_values[1][fp_index],
                X_transformed[fp_index], feature_names=feature_names)

shap.force_plot(explainer.expected_value[1], shap_values[1][fn_index],
                X_transformed[fn_index], feature_names=feature_names)
